In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('world_cup_penalty_data.csv')

In [5]:
df.head()

,game_ID,year,host_country,stage,team,pre_penalties_score,won,winner_took_first_penalty,penalty_round,penalty_scored
0,1,1982,Spain,Semi-finals,West Germany,3,True,False,1,True
1,2,1986,Mexico,Quarter-finals,France,1,True,False,1,True
2,3,1986,Mexico,Quarter-finals,West Germany,0,True,True,1,True
3,4,1986,Mexico,Quarter-finals,Belgium,1,True,False,1,True
4,5,1990,Italy,Second round,Republic of Ireland,0,True,False,1,True


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 10 columns):
game_ID                      279 non-null int64
year                         279 non-null int64
host_country                 279 non-null object
stage                        279 non-null object
team                         279 non-null object
pre_penalties_score          279 non-null int64
won                          279 non-null bool
winner_took_first_penalty    279 non-null bool
penalty_round                279 non-null int64
penalty_scored               279 non-null bool
dtypes: bool(3), int64(4), object(3)
memory usage: 16.2+ KB


In [17]:
x = list(df.penalty_round.unique())

In [18]:
type(x)

list

In [19]:
x

[1, 2, 3, 4, 5, 6]

In [28]:
def firstOrSecondKickInRound(row):
    if (row.won and row.winner_took_first_penalty) \
    or (not row.won and not row.winner_took_first_penalty):
        return 1
    else:
        return 2
    
        

In [29]:
df['kick_in_round'] = df.apply(firstOrSecondKickInRound, axis=1)
df

,game_ID,year,host_country,stage,team,pre_penalties_score,won,winner_took_first_penalty,penalty_round,penalty_scored,kick_in_round
0,1,1982,Spain,Semi-finals,West Germany,3,True,False,1,True,2
1,2,1986,Mexico,Quarter-finals,France,1,True,False,1,True,2
2,3,1986,Mexico,Quarter-finals,West Germany,0,True,True,1,True,1
3,4,1986,Mexico,Quarter-finals,Belgium,1,True,False,1,True,2
4,5,1990,Italy,Second round,Republic of Ireland,0,True,False,1,True,2
5,6,1990,Italy,Quarter-finals,Argentina,0,True,True,1,True,1
6,7,1990,Italy,Quarter-finals,Argentina,1,True,False,1,True,2
7,8,1990,Italy,Quarter-finals,West Germany,1,True,False,1,True,2
8,9,1994,USA,Second round,Bulgaria,1,True,False,1,False,2
9,10,1994,USA,Quarter-finals,Sweden,2,True,True,1,False,1


In [33]:
sum(df.penalty_scored) / len(df)

0.70250896057347667

In [38]:
df[df.penalty_scored][df.kick_in_round==2]

C:\Users\magar\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,game_ID,year,host_country,stage,team,pre_penalties_score,won,winner_took_first_penalty,penalty_round,penalty_scored,kick_in_round
0,1,1982,Spain,Semi-finals,West Germany,3,True,False,1,True,2
1,2,1986,Mexico,Quarter-finals,France,1,True,False,1,True,2
3,4,1986,Mexico,Quarter-finals,Belgium,1,True,False,1,True,2
4,5,1990,Italy,Second round,Republic of Ireland,0,True,False,1,True,2
6,7,1990,Italy,Quarter-finals,Argentina,1,True,False,1,True,2
7,8,1990,Italy,Quarter-finals,West Germany,1,True,False,1,True,2
14,15,2002,Korea/Japan,Second round,Spain,1,True,False,1,True,2
24,25,2014,Brazil,Quarter-finals,Netherlands,0,True,False,1,True,2
25,26,2014,Brazil,Semi-finals,Argentina,0,True,False,1,True,2
26,27,2018,Russia,Second round,Russia,1,True,False,1,True,2


In [39]:
df

,game_ID,year,host_country,stage,team,pre_penalties_score,won,winner_took_first_penalty,penalty_round,penalty_scored,kick_in_round
0,1,1982,Spain,Semi-finals,West Germany,3,True,False,1,True,2
1,2,1986,Mexico,Quarter-finals,France,1,True,False,1,True,2
2,3,1986,Mexico,Quarter-finals,West Germany,0,True,True,1,True,1
3,4,1986,Mexico,Quarter-finals,Belgium,1,True,False,1,True,2
4,5,1990,Italy,Second round,Republic of Ireland,0,True,False,1,True,2
5,6,1990,Italy,Quarter-finals,Argentina,0,True,True,1,True,1
6,7,1990,Italy,Quarter-finals,Argentina,1,True,False,1,True,2
7,8,1990,Italy,Quarter-finals,West Germany,1,True,False,1,True,2
8,9,1994,USA,Second round,Bulgaria,1,True,False,1,False,2
9,10,1994,USA,Quarter-finals,Sweden,2,True,True,1,False,1


In [71]:
kickByRoundDf = df.pivot_table(index=['game_ID', 'penalty_round'], columns='kick_in_round', values='penalty_scored')
kickByRoundDf.head(15)

kick_in_round              1      2
game_ID penalty_round              
1       1               True   True
        2               True   True
        3               True  False
        4              False   True
        5               True   True
        6              False   True
2       1              False   True
        2               True   True
        3               True   True
        4               True  False
        5              False   True
3       1               True   True
        2               True  False
        3               True  False
        4               True   None

In [167]:
#check df.mean doesn't include na rows in calculation
#should get same result both ways
print(kickByRoundDf[2].sum() / len(kickByRoundDf.dropna()))
print(kickByRoundDf[2].mean())

0.6992481203007519
0.699248120301


In [113]:
print("Probability of first kick of round scoring: {0:.1f}%".format(kickByRoundDf[1].mean()*100))
print("Probability of second kick of round scoring: {0:.1f}%".format(kickByRoundDf[2].mean()*100))

Probability of first kick of round scoring: 70.5%
Probability of second kick of round scoring: 69.9%


In [198]:
#dropping na rows as for the last round the second kick won't always
#have happened so can't use that row to compare
print("Probability of second kick scoring after first scored: {0:.1f}%".format(
    kickByRoundDf[kickByRoundDf[1]==True][2].mean()*100))
print("Probability of second kick scoring after first missed: {0:.1f}%".format(
    kickByRoundDf[kickByRoundDf[1]==False][2].mean()*100))

Probability of second kick scoring after first scored: 71.6%
Probability of second kick scoring after first missed: 65.8%


In [204]:
#remove multi-indexes so df is easier to iterate through
kickByRoundDf.reset_index(level=kickByRoundDf.index.names, inplace=True)

In [205]:
#get result from preceding 2nd kick on same row as first kicks
prevKickResults = []
for i in range(len(test)):
    if test.iloc[i].penalty_round == 1:
        prevKickResults.append(np.nan)
    else:
        prevKickResults.append(test.iloc[i-1][2])
        
prevKickResults

[nan,
 True,
 True,
 False,
 True,
 True,
 nan,
 True,
 True,
 True,
 False,
 nan,
 True,
 False,
 False,
 nan,
 True,
 True,
 True,
 True,
 nan,
 True,
 True,
 True,
 True,
 nan,
 False,
 True,
 True,
 False,
 nan,
 True,
 True,
 True,
 True,
 nan,
 True,
 True,
 True,
 True,
 nan,
 False,
 True,
 True,
 nan,
 True,
 True,
 True,
 False,
 True,
 nan,
 False,
 True,
 True,
 True,
 nan,
 True,
 False,
 True,
 True,
 nan,
 True,
 False,
 True,
 True,
 nan,
 True,
 True,
 False,
 nan,
 True,
 True,
 False,
 False,
 nan,
 True,
 True,
 True,
 False,
 nan,
 False,
 False,
 False,
 nan,
 True,
 False,
 True,
 nan,
 False,
 True,
 False,
 False,
 nan,
 True,
 False,
 True,
 True,
 nan,
 True,
 True,
 False,
 True,
 nan,
 True,
 True,
 False,
 False,
 nan,
 False,
 False,
 True,
 True,
 nan,
 True,
 True,
 True,
 False,
 nan,
 True,
 True,
 True,
 True,
 nan,
 True,
 True,
 True,
 nan,
 True,
 True,
 True,
 True,
 nan,
 False,
 True,
 True,
 False,
 nan,
 True,
 True,
 False,
 True,
 nan,
 Tru

In [206]:
kickByRoundDf['preceding_2nd_kick_result'] = prevKickResults
kickByRoundDf.head(15)

kick_in_round,game_ID,penalty_round,1,2,preceding_2nd_kick_result
0,1,1,True,True,NaN
1,1,2,True,True,True
2,1,3,True,False,True
3,1,4,False,True,False
4,1,5,True,True,True
5,1,6,False,True,True
6,2,1,False,True,NaN
7,2,2,True,True,True
8,2,3,True,True,True
9,2,4,True,False,True


In [207]:
print("Probability of first kick scoring after the previous round's second kick scored: {0:.1f}%".format(
    kickByRoundDf[kickByRoundDf.preceding_2nd_kick_result==True][1].mean()*100))
print("Probability of first kick scoring after the previous round's second kick missed: {0:.1f}%".format(
    kickByRoundDf[kickByRoundDf.preceding_2nd_kick_result==False][1].mean()*100))

Probability of first kick scoring after the previous round's second kick scored: 69.9%
Probability of first kick scoring after the previous round's second kick missed: 69.7%
